Dependencies

In [ ]:
!pip install graphviz
!pip install pandas

In [1]:
import graphviz
import pandas as pd
import re
from rolestereotype import RoleStereotype as rs
from collections import Counter
from collections import defaultdict
import classdiagram as cd

The Pattern class (loads .pattern DP-CORE format)

In [2]:
class Pattern:
    abstractionMapping = {
        'Normal': None,
        'Abstract': 'abstract',
        'Interface': 'interface',
        'Abstracted': 'abstracted',
        'Any': None
    }
    def __init__(self, lines):
        idx = 0
        self.name = lines[idx].strip()
        self.members = list()
        self.connections = list()
        idx += 1
        line = lines[idx]
        while line.strip() != 'End_Members':
            line = line.split()
            self.members.append(cd.Member(name=line[0], annot=Pattern.abstractionMapping[line[1]], classname=' '.join(line[2:])))
            idx += 1
            line = lines[idx]
        idx += 1
        line = lines[idx]
        while line.strip() != 'End_Connections':
            line = line.split()
            self.connections.append(cd.Connection(line[1], (line[0], line[2])))
            idx += 1
            line = lines[idx]
    def graph(self):
        dot = cd.graph(self.name, self.members, self.connections)
        return dot



Load pattern descriptions

Note: current best representation of patterns:

- bridge: bridge-v1
- adapter: object-adapter
- factory method: factory-method-v3
- abstract factory: (unknown)

In [3]:
patternNames = [
    'abstract-factory-v1', 
    'abstract-factory-v2',
    'bridge-v1',
    'bridge-v2', 
    'builder', 
    'class-adapter',
    'command', 
    'composite',
    'decorator',
    'factory-method-v1',
    'factory-method-v2',
    'factory-method-v3',
    'object-adapter',
    'observer', 
    'proxy',
    'visitor'
]

patterns = dict()
for name in patternNames:
    file = open(f'pattern-descriptions/{name}.pattern', 'r')
    lines = file.readlines()
    pattern = Pattern(lines)
    file.close()
    patterns[name] = pattern
    graph = pattern.graph()
    print(graph.source)
    # graph.render(outfile=f'pattern-descriptions/graphs/{name}.png')

digraph "Abstract Factory v1" {
	A [label=<{Concrete Factory|}> shape=record]
	B [label=<{«abstracted» Abstract Factory|}> shape=record]
	C [label=<{Product|}> shape=record]
	D [label=<{«abstracted» Abstract Product|}> shape=record]
	B -> A [arrowtail=empty dir=back]
	D -> C [arrowtail=empty dir=back]
	A -> C [arrowtail=diamond dir=back label=creates]
	A -> C [arrowhead=vee label=uses style=dashed]
}

digraph "Abstract Factory v2" {
	A [label=<{Concrete Factory|}> shape=record]
	B [label=<{«abstracted» Abstract Factory|}> shape=record]
	C [label=<{Product|}> shape=record]
	D [label=<{«abstracted» Abstract Product|}> shape=record]
	B -> A [arrowtail=empty dir=back]
	D -> C [arrowtail=empty dir=back]
	A -> C [arrowtail=diamond dir=back label=creates]
	A -> D [arrowhead=vee label=uses style=dashed]
}

digraph "Bridge v1" {
	A [label=<{Refined Abstraction|}> shape=record]
	B [label=<{«abstracted» Abstraction|}> shape=record]
	C [label=<{Concrete Implementor|}> shape=record]
	D [label=<{«ab

Load ground truth data, containing class types (e.g., abstract? interface?) and role stereotypes

In [4]:
gt = pd.read_csv('role-stereotypes.csv')

gt_k9 = gt[gt['case_name'] == 'K-9 Mail']

gt_k9[gt_k9['classname'] == 'AuthType'].iloc[0]

def getGroundTruth(classname, dframe):
    return dframe[dframe['classname'] == classname]

def isInGroundTruth(classname, dframe):
    return getGroundTruth(classname, dframe).shape[0] > 0

def roleStereotype(classname, dframe):
    return rs.from_str(getGroundTruth(classname, dframe).iloc[0]['label']) if isInGroundTruth(classname, dframe) else None

def abstraction(classname, dframe):
    gt = getGroundTruth(classname, dframe)
    if gt.shape[0] == 0:
        return None
    if gt.iloc[0]['isInterface']:
        return 'interface'
    elif gt.iloc[0]['isAbstract']:
        return 'abstract'
    else:
        return 'concrete'

print(isInGroundTruth('AuthType', gt_k9))
print(isInGroundTruth('Hello', gt_k9))

print(roleStereotype('AuthType', gt_k9), abstraction('AuthType', gt_k9))

gt_k9[gt_k9['classname'].str.contains('Builder')]

True
False
Information Holder concrete


,case_name,case_url,version_tag,full_classname,classname,isStaticClass,isInterface,isInnerClass,isClass,isEnum,classPublicity,isAbstract,label
111,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.mail.store.imap.UidSearchCommandBu...,UidSearchCommandBuilder,False,False,False,True,False,default,False,Information Holder
146,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.mail.internet.MimeMessage$MimeMess...,MimeMessage$MimeMessageBuilder,False,False,True,True,False,private,False,Structurer
304,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.search.SqlQueryBuilder,SqlQueryBuilder,False,False,False,True,False,public,False,Service Provider
434,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.message.IdentityHeaderBuilder,IdentityHeaderBuilder,False,False,False,True,False,public,False,Service Provider
436,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.message.MessageBuilder$Callback,MessageBuilder$Callback,False,True,True,False,False,public,False,Service Provider
437,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.message.MessageBuilder,MessageBuilder,False,False,False,True,False,public,True,Service Provider
438,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.message.PgpMessageBuilder,PgpMessageBuilder,False,False,False,True,False,public,False,Service Provider
440,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.message.SimpleMessageBuilder,SimpleMessageBuilder,False,False,False,True,False,public,False,Service Provider
442,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.message.TextBodyBuilder,TextBodyBuilder,False,False,False,True,False,default,False,Service Provider
503,K-9 Mail,https://github.com/k9mail/k-9/,5.304,com.fsck.k9.mailstore.MimePartStreamParser$Par...,MimePartStreamParser$PartBuilder,True,False,True,True,False,private,False,Structurer


Load DP-CORE results on K-9 Mail case

In [5]:
class PatternInstance:
    def __init__(self, pattern: Pattern, mapping):
        self._pattern = pattern
        self._mapping = mapping
        self.members = [mapping[m.name] for m in pattern.members]
        self.name='-'.join(m.name for m in self.members)
        self.connections = [cd.Connection(conn.name, tuple(mapping[p].name for p in conn.participants)) for conn in pattern.connections]
    def graph(self):
        dot = cd.graph(self.name, self.members, self.connections)
        return dot




In [6]:

def cleanup(str):
    return ''.join(c for c in str if c.isalnum())

abstractionMapping = {
    'concrete': None,
    'abstract': 'abstract',
    'interface': 'interface'
}
for name in patternNames:
    pattern = patterns[name]
    file = open(f'k-9/patterns/detect_{name}_pattern_in_k-9-5.304_project.txt', 'r')
    lines = file.readlines()
    amount = int(lines[0].split(':')[-1])
    numMembers = len(pattern.members)
    instances = [[tuple(t.strip() for t in re.split('[\(\):]', s) if t) for s in lines[3 + (i*(2+numMembers)):3 + (i*(2+numMembers))+numMembers]] for i in range(amount)]
    instances = [i for i in instances if all(isInGroundTruth(x[2], gt_k9) for x in i)]
    print(f'{pattern.name}, amount={len(instances)}')
    lst1 = []
    lst2 = defaultdict(list)
    for instance in instances:
        memberMapping = {m[0]:cd.Member(annot=abstractionMapping[abstraction(m[2], gt_k9)], name=cleanup(m[2]), classname=m[2], stereotype=roleStereotype(m[2], gt_k9)) for m in instance}
        # print(f'- {instance}')
        iname = '-'.join(x[2] for x in instance)
        pi = PatternInstance(pattern, memberMapping)
        roles = {cleanup(i[2]):f'{i[1]}:{str(roleStereotype(i[2], gt_k9))}' for i in instance}
        patt1 = ','.join(roles[m.name] for m in pi.members)
        lst1.append(patt1)
        for r in roles:
            p,s = tuple(roles[r].split(':'))
            lst2[p].append(s)
        # graph = pi.graph()
        # print(graph.source)
        # graph.render(outfile=f'k-9/patterns/graphs/{name}/{iname}.png')
        
    print('- pattern instances')
    ctr1 = Counter(lst1)
    for e,c in ctr1.most_common():
        print(f"  - {c}: {e}")

    print('- participants')
    for p in lst2:
        print(f"  - {p}")
        ctr2 = Counter(lst2[p])
        for e,c in ctr2.most_common():
            print(f"    - {c}: {e}")


    file.close()

Abstract Factory v1, amount=1
- pattern instances
  - 1: Concrete Factory:Coordinator,Abstract Factory:Information Holder,Product:Structurer,Abstract Product:Information Holder
- participants
  - Concrete Factory
    - 1: Coordinator
  - Abstract Factory
    - 1: Information Holder
  - Product
    - 1: Structurer
  - Abstract Product
    - 1: Information Holder
Abstract Factory v2, amount=2
- pattern instances
  - 1: Concrete Factory:Service Provider,Abstract Factory:Service Provider,Product:Information Holder,Abstract Product:Service Provider
  - 1: Concrete Factory:Structurer,Abstract Factory:Structurer,Product:Service Provider,Abstract Product:Service Provider
- participants
  - Concrete Factory
    - 1: Service Provider
    - 1: Structurer
  - Abstract Factory
    - 1: Service Provider
    - 1: Structurer
  - Product
    - 1: Information Holder
    - 1: Service Provider
  - Abstract Product
    - 2: Service Provider
Bridge v1, amount=4
- pattern instances
  - 1: Refined Abstraction

Check role stereotype relationships

In [7]:
names = {
    'calls',
    'creates',
    'has',
    'inherits',
    'references',
    'uses'
}

m_g = set()
c_g = list()

for name in names:
    file = open(f'connection-descriptions/{name}.pattern', 'r')
    lines = file.readlines()
    file.close()
    pattern = Pattern(lines)

    file = open(f'k-9/connections/detect_{name}_pattern_in_k-9-5.304_project.txt', 'r')
    lines = file.readlines()
    file.close()
    amount = int(lines[0].split(':')[-1])
    numMembers = len(pattern.members)
    instances = [[tuple(t.strip() for t in re.split('[\(\):]', s) if t) for s in lines[3 + (i*(2+numMembers)):3 + (i*(2+numMembers))+numMembers]] for i in range(amount)]
    instances = [i for i in instances if all(isInGroundTruth(x[2], gt_k9) for x in i)]
    print(f'{pattern.name}, amount={len(instances)}')

    lst1 = []
    lst2 = []

    members = set()
    connections = list()

    for instance in instances:
        # print(f'- {instance}')
        memberMapping = {m[0]:cd.Member(annot=abstraction(m[2], gt_k9), name=cleanup(m[2]), classname=m[2], stereotype=roleStereotype(m[2], gt_k9)) for m in instance}
        pi = PatternInstance(pattern, memberMapping)
        roles = {cleanup(i[2]):str(roleStereotype(i[2], gt_k9)) for i in instance}
        patt1 = ','.join(f' {c.name} '.join(roles[p] for p in c.participants) for c in pi.connections)
        lst1.append(patt1)
        # lst2.append(patt2) 
        # graph = pi.graph()
        # print(graph.source)
        # graph.render(outfile=f'k-9/patterns/graphs/{name}/{iname}.png')
        members.update(pi.members)
        connections += pi.connections
        
    ctr1 = Counter(lst1)
    # ctr2 = Counter(lst2)

    l = len("Information Holder")
    for e,c in ctr1.most_common():
        ee = e.split(f" {name} ")
        print(f"  {ee[0]:>{l}} {name} {ee[1]:<{l}}: {c}")

    # l = len("interface:Information Holder")
    # for e,c in ctr2.most_common():
    #     ee = e.split(" extends ")
    #     print(f"{ee[0]:>{l}} extends {ee[1]:<{l}}: {c}")

    prt = [c.participants for c in connections]
    a,b = tuple(zip(*prt))
    ctra = Counter(a)
    ctrb = Counter(b)
    
    print('  Participant A')
    for e,c in ctra.most_common(5):
        print(f"  -- {e}: {c}")
        
    print('  Participant B')
    for e,c in ctrb.most_common(5):
        print(f"  -- {e}: {c}")

    m_g.update(members)
    c_g += connections

    # g = cd.graph(name, members, connections)
    # g.render(outfile=f'k-9/connections/graphs/{name}.png')



Uses, amount=159
    Service Provider uses Information Holder: 30
  Information Holder uses Information Holder: 23
    Service Provider uses Service Provider  : 21
          Structurer uses Information Holder: 14
          Interfacer uses Information Holder: 10
          Structurer uses Structurer        : 9
    Service Provider uses Structurer        : 8
          Structurer uses Service Provider  : 8
  Information Holder uses Structurer        : 5
  Information Holder uses Service Provider  : 5
          Controller uses Information Holder: 4
         Coordinator uses Structurer        : 4
         Coordinator uses Information Holder: 3
          Interfacer uses Coordinator       : 2
          Interfacer uses Service Provider  : 2
          Controller uses Service Provider  : 2
    Service Provider uses Controller        : 1
          Interfacer uses Structurer        : 1
          Interfacer uses Controller        : 1
          Controller uses Structurer        : 1
    Service Provid

In [8]:
print('# MEMBERS #########################################################################################')
for m in m_g:
    print(f'{m.classname},{m.annot},{str(m.stereotype)}')
print('# CONNECTIONS #####################################################################################')
for c in c_g:
    print(f'{c.name},{c.participants[0]},{c.participants[1]}')

# MEMBERS #########################################################################################
Multipart,abstract,Structurer
TracingPowerManager,concrete,Service Provider
CoreService,abstract,Service Provider
Part,interface,Service Provider
EncryptionDetector,concrete,Service Provider
BaseAccount,interface,Information Holder
UnreadWidgetProperties,concrete,Coordinator
MessageFulltextCreator,concrete,Service Provider
Account,concrete,Information Holder
MessageFulltextCreator,concrete,Service Provider
TextPartFinder,concrete,Service Provider
NotificationActionService,concrete,Controller
MessageFulltextCreator,concrete,Service Provider
UnreadWidgetConfiguration,concrete,Interfacer
AdvancedNodeTraversor,concrete,Service Provider
UnreadWidgetConfiguration,concrete,Interfacer
BodyPart,abstract,Information Holder
LauncherShortcuts,concrete,Interfacer
SearchAccount,concrete,Coordinator
LauncherShortcuts,concrete,Interfacer
HtmlSignatureRemover,concrete,Coordinator
Multipart,abstract,Struc